In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/IPL/Final_Experiment2/* /content/

# Dependencies

## Image Classification

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import pipeline
from PIL import Image
import requests

checkpoint = "openai/clip-vit-large-patch14"
classifier = pipeline(model=checkpoint, task="zero-shot-image-classification")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor confi

In [ ]:
def find_label(label):
  if label in ["animal", "pet"]:
    return "animal", 1
  elif label in ["landscape", "city", "flower"]:
    return "landscape", 2
  elif label in ["sport"]:
    return "sport", 3
  elif label in ["chat", "book"]:
    return "text", 4
  elif label in ["art", "painting", "statue"]:
    return "art", 5
  elif label in ["instrument"]:
    return "object", 6
  elif label in ["food", "cafe", "restaurant"]:
    return "food", 7
  elif label in ["people"]:
    return "people", 8
  elif label in ["movie", "animation"]:
    return "movie", 9
  elif label in ["music"]:
    return "music", 10
  elif label in ["shop"]:
    return "shop", 11
  return "other", 12

## Face Recognition

In [ ]:
!pip3 install face_recognition
!git clone https://github.com/davisking/dlib.git
%cd dlib
!pip install cmake
!mkdir build
%cd build
!cmake ..; cmake --build .
%cd ..
!python3 setup.py install
%cd ..
import shutil

In [ ]:
from PIL import Image
import os
import face_recognition

def face_detection(profile, media_pic, media_id, id, mediaset):
  # print(media_pic)
  picture_of_me = face_recognition.load_image_file(profile)
  my_face_encoding = face_recognition.face_encodings(picture_of_me)[0]
  unknown_picture = face_recognition.load_image_file(media_pic)
  # unknown_face_encoding = face_recognition.face_encodings(unknown_picture)[0]
  # results = face_recognition.compare_faces([my_face_encoding], unknown_face_encoding)

  face_locations = face_recognition.face_locations(unknown_picture)
  image = unknown_picture
  # print("I found {} face(s) in this photograph.".format(len(face_locations)))

  for face_location in face_locations:

      # Print the location of each face in this image
      top, right, bottom, left = face_location
      # print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

      # You can access the actual face itself like this:
      face_image = image[top:bottom, left:right]
      pil_image = Image.fromarray(face_image)
      _ = pil_image.save(f'{id}/media/media{id}-{media_id}.jpg')
      face = face_recognition.load_image_file(f'{id}/media/media{id}-{media_id}.jpg')
      if len(face_recognition.face_encodings(face)) == 0:
        os.remove(f'{id}/media/media{id}-{media_id}.jpg')
        continue
      else:
        mediaset.add(media_pic)
      face = face_recognition.face_encodings(face)[0]
      # print("face_recognition.compare_faces([my_face_encoding], face)", face_recognition.compare_faces([my_face_encoding], face))
      if not face_recognition.compare_faces([my_face_encoding], face)[0]:
        os.remove(f'{id}/media/media{id}-{media_id}.jpg')
      else:
        mediaset.add(media_pic)
        # print("yes")
      pil_image = Image.fromarray(face_image)
      pil_image.show()
      media_id += 1
  return media_id, mediaset

## Facial Expression Analysis

In [ ]:
!pip install timm

In [ ]:
import timm

# model_reloaded = timm.create_model('schibsted/Facial_Recognition_with_Sentiment_Detector', pretrained=True)

In [ ]:
# Facial expression classifier
import os
from fastai.vision.all import *

# Emotion
learn_emotion = load_learner('emotions_vgg19.pkl')
learn_emotion_labels = learn_emotion.dls.vocab

# Sentiment
# learn_sentiment = load_learner('sentiment_vgg19.pkl')
# learn_sentiment_labels = learn_sentiment.dls.vocab

# Predict
def predict(img):
    img = PILImage.create(img)

    pred_emotion, pred_emotion_idx, probs_emotion = learn_emotion.predict(img)

    # pred_sentiment, pred_sentiment_idx, probs_sentiment = learn_sentiment.predict(img)

    #emotions = {f'emotion_{learn_emotion_labels[i]}': float(probs_emotion[i]) for i in range(len(learn_emotion_labels))}
    #sentiments = {f'sentiment_{learn_sentiment_labels[i]}': float(probs_sentiment[i]) for i in range(len(learn_sentiment_labels))}

    emotions = {learn_emotion_labels[i]: float(probs_emotion[i]) for i in range(len(learn_emotion_labels))}
    # sentiments = {learn_sentiment_labels[i]: float(probs_sentiment[i]) for i in range(len(learn_sentiment_labels))}

    return [emotions] #{**emotions, **sentiments}

# Image Classification

In [ ]:
def image_classification(id, files_list):
  ic = ""
  for i in range(len(files_list)):
    image = Image.open(f'{id}/{files_list[i]}')
    predictions = classifier(image, candidate_labels=["animal", "pet", "landscape", "sport", "chat", "art", "instrument", "food", "people", "movie", "music", "shop", "cafe", "restaurant", "city", "painting", "statue", "book", "animation", "flower"])
    print(find_label(predictions[0]['label'])[1], end="#")
    ic += str(find_label(predictions[0]['label'])[1])
    ic += "#"
  print()
  return ic

# Face Recognition

In [ ]:
def face_recognition_module(id, files_list):
  media_id = 0
  mediaset = set()
  if os.path.isdir(f"{id}/media"):
    shutil.rmtree(f"{id}/media")
  os.mkdir(f"{id}/media")
  for i in files_list:
    if i != "profile.jpeg":
      media_id, mediaset = face_detection(f"{id}/profile.jpeg", f"{id}/{i}", media_id, id, mediaset)

  media_list = [f for f in listdir(join(id, "media")) if isfile(join(join(id, "media"), f))]
  print(mediaset)
  print(len(media_list)/(len(files_list)-1))

# Facial Expression Analysis

In [ ]:
def facial_expression_analysis(id):
  expressions = predict(f"{id}/profile.jpeg")[0]
  expressions["angry"] *= 3
  expressions["disgust"] *= 3
  expressions["fear"] *= 3
  expressions["happy"] *= 3
  expressions["neutral"] *= 3
  expressions["sad"] *= 3
  expressions["surprise"] *= 3
  media_list = [f for f in listdir(join(id, "media")) if isfile(join(join(id, "media"), f))]
  for i in media_list:
    exp = predict(f"{id}/media/{i}")[0]
    expressions["angry"] += exp["angry"]
    expressions["disgust"] += exp["disgust"]
    expressions["fear"] += exp["fear"]
    expressions["happy"] += exp["happy"]
    expressions["neutral"] += exp["neutral"]
    expressions["sad"] += exp["sad"]
    expressions["surprise"] += exp["surprise"]

  expressions["angry"] /= (len(media_list)+3)
  expressions["disgust"] /= (len(media_list)+3)
  expressions["fear"] /= (len(media_list)+3)
  expressions["happy"] /= (len(media_list)+3)
  expressions["neutral"] /= (len(media_list)+3)
  expressions["sad"] /= (len(media_list)+3)
  expressions["surprise"] /= (len(media_list)+3)
  print(expressions)
  return expressions


# Test

In [ ]:
# topics = "6#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9"

def format_topics(topics):
  reshte = topics.split("#")
  reshte.pop()
  print(reshte)

  def labels(label):
    if label == "animal":
      return 1
    elif label == "landscape":
      return 2
    elif label == "sport":
      return 3
    elif label == "text":
      return 4
    elif label == "art":
      return 5
    elif label == "object":
      return 6
    elif label == "food":
      return 7
    elif label == "people":
      return 8
    elif label == "movie":
      return 9
    elif label == "music":
      return 10
    elif label == "shop":
      return 11
    return 12

  results = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  count_reshte = 0
  for i in reshte:
    # if i == '':
    #   continue
    count_reshte += 1
    results[int(i)-1] += 1


  for r in range(len(results)):
    results[r] /= count_reshte
    results[r] *= 100
    results[r] = str(int(results[r]))

  return "#".join(results)

# print(format_topics(topics))

In [ ]:
# sentiment = {'angry': 0.015877289033142006, 'disgust': 0.012511020366478729, 'fear': 0.14441619933183705, 'happy': 0.27127157151699066, 'neutral': 0.11558954133570533, 'sad': 0.02055548384226848, 'surprise': 0.4197788978261607}

def format_sentiment(sentiment):
  sent = ""
  sent += (str(round(sentiment["happy"], 4)) + "#")
  sent += (str(round(sentiment["sad"], 4)+round(sentiment["fear"], 4)) + "#")
  sent += (str(round(sentiment["angry"], 4)+round(sentiment["disgust"], 4)) + "#")
  sent += (str(round(sentiment["neutral"], 4)) + "#")
  sent += str(round(sentiment["surprise"], 4))
  return sent

# print(format_sentiment(sentiment))

In [ ]:
from os import listdir
from os.path import isfile, join
ids_lst = [
# "supsnm",
# "mmsamiei97",
# "sarative",
# "Masihbr",
# "SElahimanesh",
# "mohammadali_79",
# "iTs_kAsraa",
# "alizademhdi",
# "majidtaherkhan5",
# "Iman_M_02",
# "MojoodeZende",
# "lellahka",
# "Donya_rz",
# "RFzarghani",
# "Kahbye",
# "amirhbrt",
# "Thy_Crow",
# "lonelyrtalking",
# "itszahraaw",
# "alilordloss",
# "pahdemy",
# "arearevalla",
# "the_shabe",
# "A_zoje",
# "Amirzgh1",
# "GhermezzzMohi",
# "Zhra_arj",
# "SuccessfulEhsan",
# "Aelirus",
# "Sqrtpiee",
# "WhatYouSeeIsMee",
# "aexomir1",
# "Esmailysetayesh",
# "Shepelll",
# "nottheoneuknow_",
# "Haamedht",
# "ItisHoor",
# "Necessiite",
# "BirdSuspicious",
# "Yganeyzdi",
# "MguidedS",
# "Atenanourii",
# "Ehsanino82",
# "___mamali___",
# "MGhasemi8156",
# "BardiaWasHere",
# "seginus14",
# "Smmhatami",
# "OneToBeShunned",
# "Hamriouz",
# "amirhossein_nr",
# "maghasemzadeh",
# "MehradMilan",
# "hosseinkhaniii",
# "AmirrezaFN2000",
# "BardiaT3",
# "aminam80",
# "Mr_mahdimirzaei",
# "FatemehAsgarii",
# "rez1pm",
# "mahdiakhi",
# "Zhugulibu",
# "SMahdiFaghih",
# "its_Hedy",
# "Thebigwhoman",
# "atalmataltotol"
# "ReallyNemidanam",
# "GhermezzzMohi"
# "aidinhnkhn",
# "imligeia",
# "Aliredaq",
# "cabrrrrrrron",
# "sharif0108",
# "slmktbr"
"pouyalahabi"
]

In [ ]:
def run_test(id):
  files_list = [f for f in listdir(id) if isfile(join(id, f))]
  ##
  lst = []
  for i in files_list:
    if i.replace(".jpeg", "").isdigit():
      lst.append(int(i.replace(".jpeg", "")))
  lst.sort()
  files_list = []
  for i in lst:
    files_list.append(f"{i}.jpeg")
  ##
  print(id, len(files_list), files_list)
  # print(files_list)
  print("TOPICS", format_topics(image_classification(id, files_list)))
  try:
    face_recognition_module(id, files_list)
  except:
    print("face_recognition not possible!")
  print("EXPRESSIONS", format_sentiment(facial_expression_analysis(id)))

In [ ]:
for i in ids_lst:
  run_test(i)

pouyalahabi 20 ['1.jpeg', '2.jpeg', '3.jpeg', '4.jpeg', '5.jpeg', '6.jpeg', '7.jpeg', '8.jpeg', '9.jpeg', '10.jpeg', '11.jpeg', '12.jpeg', '13.jpeg', '14.jpeg', '15.jpeg', '16.jpeg', '17.jpeg', '18.jpeg', '19.jpeg', '20.jpeg']
8#9#4#9#9#9#4#4#4#2#8#8#8#8#2#1#5#8#8#8#
['8', '9', '4', '9', '9', '9', '4', '4', '4', '2', '8', '8', '8', '8', '2', '1', '5', '8', '8', '8']
TOPICS 5#10#0#20#5#0#0#40#20#0#0
{'pouyalahabi/11.jpeg', 'pouyalahabi/14.jpeg', 'pouyalahabi/13.jpeg', 'pouyalahabi/17.jpeg', 'pouyalahabi/3.jpeg', 'pouyalahabi/18.jpeg', 'pouyalahabi/20.jpeg'}
0.3157894736842105


{'angry': 0.10982356367296436, 'disgust': 0.02201911976347522, 'fear': 0.03574925469648507, 'happy': 0.3578552446431584, 'neutral': 0.1172441607842403, 'sad': 0.06436336646179876, 'surprise': 0.2929453342739079}
EXPRESSIONS 0.3579#0.1001#0.1318#0.1172#0.2929


# Evaluation

In [ ]:
import csv

topics = "6#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9#9"
reshte = topics.split("#")
id = "mr_hooomed"

def labels(label):
  if label == "animal":
    return 1
  elif label == "landscape":
    return 2
  elif label == "sport":
    return 3
  elif label == "text":
    return 4
  elif label == "art":
    return 5
  elif label == "object":
    return 6
  elif label == "food":
    return 7
  elif label == "people":
    return 8
  elif label == "movie":
    return 9
  elif label == "music":
    return 10
  elif label == "shop":
    return 11
  return 12

this_id = id
flag = 0
deghat = 0
counter = -2
with open('labels.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
      if ",media" in row:
        break
      if row[0].split(",")[0] == this_id:
        counter += 1
        if flag == 0:
          flag = 1
          continue
        # print(row[0].split(",")[4].split("/"), counter)
        for i in row[0].split(",")[4].split("/"):
          # print(reshte[counter], labels(i))
          if int(reshte[counter]) == labels(i):
            # print(deghat)
            deghat += 1
# print(len(reshte))
print(deghat/len(reshte))

FileNotFoundError: ignored

In [ ]:
id = "SElahimanesh"
topics = "7#7#8#11#2#4#4#2#4#4#4#4#4#2#1#11#4#11#4#8"

files_list = [f for f in listdir(id) if isfile(join(id, f))]
lst = []
for i in files_list:
  if i.replace(".jpeg", "").isdigit():
    lst.append(int(i.replace(".jpeg", "")))
lst.sort()
files_list = []
for i in lst:
  files_list.append(f"{i}.jpeg")
print(id, files_list)

# Earth Mover's Distance

In [3]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def earth_movers_distance(p, q):
    if len(p) != len(q):
        raise ValueError("Input sets must have the same length.")

    # Normalize the input sets to ensure they sum to 1
    p = np.array(p) / np.sum(p)
    q = np.array(q) / np.sum(q)

    # Calculate the cost matrix
    cost_matrix = np.abs(np.subtract.outer(p, q))

    # Solve the linear sum assignment problem to find optimal transport plan
    row_indices, col_indices = linear_sum_assignment(cost_matrix)

    # Calculate the Earth Mover's Distance
    emd = cost_matrix[row_indices, col_indices].sum()

    return emd

# Example usage:
set1 = [42.34, 2.7, 0, 46.85, 7.21]
set2 = [45.9, 8.66, 4.88, 4.48, 36.12]

distance = earth_movers_distance(set1, set2)
print("Similarity", 1-distance)


Similarity 0.8397441023590564


# KL Divergence

In [ ]:
#define two probability distributions
P = [0.1, 0.15, 0.25, 0.3, 0.2]
Q = [0.2, 0.3, 0.25, 0.15, 0.1]
from scipy.special import rel_entr

#calculate (P || Q)
sum(rel_entr(P, Q))

In [ ]:
import sklearn
sklearn.metrics.mutual_info_score(P, Q)

In [ ]:
import math
# calculate the kl divergence
def kl_divergence(p, q):
 return sum(p[i] * math.log((p[i]/q[i]), 2) for i in range(len(p)))

kl_divergence(P, Q)

In [1]:
import numpy as np

def jensen_shannon_divergence(p, q):
    if len(p) != len(q):
        raise ValueError("Input sets must have the same length.")

    # Normalize the input sets to ensure they sum to 1
    p = np.array(p) / np.sum(p)
    q = np.array(q) / np.sum(q)

    # Calculate the midpoint distribution
    m = 0.5 * (p + q)

    # Calculate the KL divergences from p and q to m
    kl_divergence_p = np.sum(p * np.log2(p / m))
    kl_divergence_q = np.sum(q * np.log2(q / m))

    # Calculate the average of the KL divergences (Jensen-Shannon Divergence)
    jensen_shannon_divergence = 0.5 * (kl_divergence_p + kl_divergence_q)

    return jensen_shannon_divergence

# Example usage:
set1 = [42.34, 2.7, 0, 46.85, 0]
set2 = [45.9, 8.66, 4.88, 4.48, 36.12]

distance = jensen_shannon_divergence(set1, set2)
print("Jensen-Shannon Divergence Similarity:", 1- distance)


Jensen-Shannon Divergence Similarity: nan


<ipython-input-1-a52dde4de1fc>:15: RuntimeWarning: divide by zero encountered in log2
  kl_divergence_p = np.sum(p * np.log2(p / m))
<ipython-input-1-a52dde4de1fc>:15: RuntimeWarning: invalid value encountered in multiply
  kl_divergence_p = np.sum(p * np.log2(p / m))


In [ ]:
import numpy as np

def euclidean_distance(p, q):
    if len(p) != len(q):
        raise ValueError("Input sets must have the same length.")

    p = np.array(p)
    q = np.array(q)

    # Calculate the Euclidean distance
    euclidean_distance = np.sqrt(np.sum((p - q)**2))

    return euclidean_distance

# Example usage:
set1 = [0.1, 0.1, 0.1, 0.1, 0.6]
set2 = [0.15, 0.25, 0.25, 0.2, 0.15]

distance = euclidean_distance(set1, set2)
print("Euclidean Distance:", 1-distance)


# Extra

In [ ]:
# !mv images/images/* ./
!rm -r "images"

In [ ]:
for i in ids_lst:
  if not os.path.exists(i):
    !mkdir {i}

In [ ]:
!jar xvf "images.zip"